In [1]:
from PIL import Image
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import numpy as np
from PIL import Image

def load_and_invert_mask(mask_path):
    mask = Image.open(mask_path).convert("L")
    mask_array = np.array(mask)
    
    inverted_mask = 255 - mask_array

    for i in range(len(inverted_mask)):
        for j in range(len(inverted_mask[i])):
            if inverted_mask[i][j] != 255:
                inverted_mask[i][j] = 0
    
    return inverted_mask

def apply_mask(image_path, mask, output_path):
    image = Image.open(image_path)
    image = image.convert("RGBA")

    alpha = np.full((image.height, image.width), 255, dtype=np.uint8)
    alpha[mask == 0] = 0

    result = Image.merge("RGBA", (image.split()[:3] + (Image.fromarray(alpha),)))

    result.save(output_path)

In [19]:
mask_img_list = os.listdir("IM_Resize/masks/")
mask_img_list = [image for image in mask_img_list if str(image)[0] != "." ]

hotels = os.listdir("IM_Resize/images/")
hotels = [hotel for hotel in hotels if str(hotel)[0] != "." ]

test_size = 0.2
val_size = 0.15
for hotel in (pbar:= tqdm(hotels)):
    df = []
    pbar.set_description(f"Indexing hotel {hotel}")
    images = os.listdir("IM_Resize/images/" + str(hotel) + "/")
    images = [image for image in images if str(image)[0] != "." ]

    count_im = 0
    for image in images:
        count_mask = 0
        for mask_img in mask_img_list:
            pbar.set_description(f"Indexing hotel {hotel} image {count_im}/{len(images) - count_im} with mask {mask_img}. \
                                  {len(mask_img_list) - count_mask} remaining")
            df.append([hotel, image, mask_img])
            count_mask += 1
        count_im += 1
    
    df = pd.DataFrame(df, columns=['Hotel', 'Image', 'Mask_Image'])
    df["Combined"] = df["Hotel"] + "/" + df["Image"] + "/" + df["Mask_Image"]
    df.to_csv("Support Files/masking_summary.csv", index=False, mode='a')

    hotel = list(df["Hotel"].unique())[0]
    hotel_images = list(df.loc[df["Hotel"] == str(hotel)]['Combined'])
    test_images = np.random.choice(hotel_images, int(len(hotel_images)*test_size), replace=False)
    train_images = list(set(hotel_images) - set(test_images))

    val_images = np.random.choice(train_images, int(len(hotel_images)*val_size), replace=False)
    train_images = list(set(train_images) - set(val_images))

    count = 0
    for c_image in train_images:
        pbar.set_description(f"Creating mask for train pair {c_image}")
        c_image = c_image.split("/")
        hotel = c_image[0]
        h_image = c_image[1]
        mask = c_image[2]

        try:
            os.mkdir("HotelTraffickingData")
        except FileExistsError:
            pass

        try:
            os.mkdir("HotelTraffickingData/train")
        except FileExistsError:
            pass

        try:
            os.mkdir("HotelTraffickingData/" + str(hotel))
        except FileExistsError:
            pass
        
        mask_path = "IM_Resize/masks/" + str(mask)
        image_path = "IM_Resize/images/" + str(h_image)
        write_path = "HotelTraffickingData/train/" + str(hotel) + str(f"{count}.png")
        
        loaded_mask = load_and_invert_mask(mask_path)
        apply_mask(image_path, loaded_mask, "masked_image.png")
        break
    break

Indexing hotel 76693 image 0/1393 with mask 00064.png.                                   4891 remaining:   0%|          | 0/30 [00:00<?, ?it/s]

Indexing hotel 76693 image 5/1388 with mask 00006.png.                                   4944 remaining:   0%|          | 0/30 [00:14<?, ?it/s]


KeyboardInterrupt: 

In [13]:
print(hotel, h_image, mask)
# df

76693 000041632.jpg 04102.png


In [11]:
print(len(hotel_images))
print(len(train_images))
print(len(val_images))
print(len(test_images))

len(train_images)+len(val_images)+len(test_images)

6895350
4481978
1034302
1379070


6895350

In [6]:
hotels = list(df["Hotel"].unique())

test_size = 0.2
val_size = 0.15
for hotel in (pbar:= tqdm(hotels)):
    images = list(df.loc[df["Hotel"] == '76693']['Combined'])
    test_images = np.random.choice(images, int(len(images)*test_size), replace=False)
    train_images = list(set(images) - set(test_images))

    val_images = np.random.choice(train_images, int(len(images)*val_size), replace=False)
    train_images = list(set(train_images) - set(val_images))

    for image in train_images:
        pbar.set_description(f"Creating mask for train pair {image}")
        image = image.split("/")
        hotel = image[0]
        h_image = image[1]
        
    break

Creating mask for train pair 76693/000041249.jpg/01056.png:   0%|          | 0/1 [00:50<?, ?it/s]


KeyboardInterrupt: 

In [28]:
train_images[0].split("/")

['76693', '000041658.jpg', '8268']

In [ ]:
mask_path = "/Volumes/Storage/Git Repos/AML/IM_Resize/masks/00000.png"
loaded_mask = load_and_invert_mask(mask_path)
apply_mask("/Volumes/Storage/Git Repos/AML/IM_Resize/images/2515/000043495.jpg", loaded_mask, "masked_image.png")
